In [5]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install keras

     ---------------------------------------- 1.7/1.7 MB 4.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install daal==2021.4.0

     ---------------------------------------- 69.0/69.0 MB 2.6 MB/s eta 0:00:00
     -------------------------------------- 283.8/283.8 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tbb
    Found existing installation: TBB 0.2
Note: you may need to restart the kernel to use updated packages.


ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [20]:
pip install numpy==1.21.6

     ---------------------------------------- 14.0/14.0 MB 3.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.21.6 which is incompatible.


# Detecting fake images

In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam

# Set the path to the directory containing the images
img_dir = r"C:\Users\Rohith\Desktop\Projects\AlcloveX intership\images_from_video_big"

# Get a list of all the image filenames in the directory
img_files = os.listdir(img_dir)

# Create empty lists to store the images and labels
images = []
labels = []

# Loop over each image and load it into memory
for img_file in img_files:
    # Load the image
    img_path = os.path.join(img_dir, img_file)
    img = cv2.imread(img_path)

    # Resize the image to a smaller size
    img = cv2.resize(img, (256, 256))

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Append the image to the list of images
    images.append(gray)
    
    # Determine the label based on the filename (assuming real images have "real" in the filename)
    if int(img_file[-5]) ==1:
        labels.append(1)
    else:
        labels.append(0)

# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Check if the dataset has at least one sample
if images.shape[0] == 0:
    print("Dataset is empty.")
    exit()

# Split the dataset into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.3, random_state=42)

# Reshape the images to match the input shape of the CNN
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

# Define the architecture of the CNN
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(256, 256, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(1, activation="sigmoid")
])
# Compile the model with an optimizer and loss function
model.compile(optimizer=Adam(learning_rate=0.01), loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(train_images, train_labels, epochs=5, batch_size=32, validation_data=(test_images, test_labels))

# Evaluate the model on the testing set
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")